# Intro to Deep Learning, aka., Neural Networks

The most essential step in learning is distinguishing two distinct groups among a given collection of objects. This is at its root a non-linear process. The very same classification we obtain entails a non-linearity: "*on one side everything is smooth and green; on the other side everything is smooth but red*". It is this process that is the gist of human learning and reasoning capabilities.

The simplest mathematical and computational toy model that is capable of making such distinctions is the **perceptron**. This is the most simple step which **all neural networks** build upon. It is in fact thus the "hello world" example of a neural network.

### Perceptron

<pre>
2 inputs                                     N inputs

X1o                                          X1o
   \                                             \
    \W1                                           \
     \             --                              \             --
      ----------  |   --------o y            X2o--------------  |   --------o y
     /          __|                          .     /          __|
    /W2                                      .    /            
X2o/                                         .   /
                                                /
         |x2                                Xno/ 
     \ + |                                    
      \  |  (w1, w2)
     - \ | /^  +
   _____\|/_________ x1
         |\
     -   | \   +
         |  \

The simple perceptron is capable of classifying points (x1, x2) (or of making distinctions) by drawing a line.
This, in effect, generates a non-linearity in our view of the universe: things on "the left" side of the line
and things on "the right" side. The line has a perpendicular given by the vector (w1, w2). The latter points
in the direction where the output y is larger in the algebraic sense (smaller if the non-linear function is an
inversion). In the case of N inputs, the perceptron classifies points in an N-dimensional space by drawing an 
(N-1)-hyperplane. For N>=3, it classifies them between those "inside" that hyperplane and those outside it.
</pre>

In [5]:
def perceptron(x1,x2,w1,w2,str=False,debug=False):
    """
    Simple perceptron w/ 2 inputs, 2 weights and Heaviside step function
    """
    z = x1*w1+x2*w2
    if debug: print("x1: ",x1,"  x2: ",x2,"  w1: ",w1,"  w2: ",w2,"  z: ",z,end=" -->  ")
    if not str: return -1 if z<0 else 1;
    else: return "-" if z<0 else "1";

print(perceptron(-0,2,0,1))

def nPerceptron(X,W,str=False):
    assert(len(X)==len(W))
    z=0;
    for i in range(0, len(X)):
        z+=X[i]*W[i]
    if not str: return -1 if z<0 else 1;
    else: return "-" if z<0 else "1";

nPerceptron([3,2,2],[0,1,2])

1


1

In [8]:
from math import floor
import numpy as np

def scan2InputNN(nn,weights,xysup=11,showHidden=None,debug=False): 
    """
    Pass a 'neural network with 2 inputs and its weights, then scan a given range of values for those inputs and see 
     where the NN classifies those points into ('-' / '+')
     
    NN needs to have defined inputs and weights as simple array. Also optional variables showHidden, 
    for showing values of hidden neurons, or debug for additional info
    
    xysup=11 #supremum of x,y ranges, with -xysup(1-[1/2]) <= x,y <= xysup(1-[1/2]). 
    E.g., xysup=11, xy \in [-5,5]
    
    """
    X1 = np.arange(0,xysup)
    X2 = np.arange(0,xysup)
    X1 -= floor(xysup/2)
    X2 -= floor(xysup/2)
    X1

    if showHidden: print("Showing hidden neuron ",showHidden)
        
    print("      ",end="") #formatting
    for x1 in X1:          #Print first line w/ x-coord 
        print(x1,end="  ")
    print("\n\t",end="") 
    for x2 in reversed(X2):                               #Print first y coor. (+ values top/first=>reverse array)
        print(x2,end=" ") if x2<0 else print(x2,end="  ")
        for x1 in X1:                                     #...then go on w/ values along x-axis
            #print(nn1HiddenLayer3([x1,x2],W),end="  " )
            #if debug: help(nn)
            print(nn([x1,x2],weights,showHidden=showHidden,debug=debug),end="  " )
        print("\n\t",end="")

"""def scan2InputPerceptron(ptron,weights,debug=False):
    def wraptron(X,weights,showHidden=None):
        ptron(X[0],X[1],weights[0],weights[1],str=True,debug=debug)
    
    scan2InputNN(wraptron,weights)
"""    
def wraptron(X,weights,showHidden=None,debug=False):
    return perceptron(X[0],X[1],weights[0],weights[1],str=True,debug=debug)


In [9]:
#scan2InputPerceptron(perceptron,weights=[0,1],debug=True)
scan2InputNN(wraptron,weights=[2,3],debug=False)
print(perceptron(-4,-3,1,1,debug=True))

      -5  -4  -3  -2  -1  0  1  2  3  4  5  
	5  1  1  1  1  1  1  1  1  1  1  1  
	4  1  1  1  1  1  1  1  1  1  1  1  
	3  -  1  1  1  1  1  1  1  1  1  1  
	2  -  -  1  1  1  1  1  1  1  1  1  
	1  -  -  -  -  1  1  1  1  1  1  1  
	0  -  -  -  -  -  1  1  1  1  1  1  
	-1 -  -  -  -  -  -  -  1  1  1  1  
	-2 -  -  -  -  -  -  -  -  1  1  1  
	-3 -  -  -  -  -  -  -  -  -  -  1  
	-4 -  -  -  -  -  -  -  -  -  -  -  
	-5 -  -  -  -  -  -  -  -  -  -  -  
	x1:  -4   x2:  -3   w1:  1   w2:  1   z:  -7 -->  -1


### Multilayer perceptron, aka., NN

<pre>
          W1                           __     h1
X1o---------------------------------  |   ------------o
   \ \              /               __|                \
    \  \           /W2                                  \
     \   \        /                                      \ 
      \    \     /                                        \
       \     \  /                                          \
        \      /\                                           \w7
         \    /  \ W3                                        \
          \  /    \                                           \
            /      \                    __     h2          w8  \           __            
           /\       /----------------  |   ------------o----------------  |   --------o y
          /  \     /                 __|                       /        __|              
         /    \   /W4                                         /
        /      \ /                                           /
       /      / \                                           /w9
      /     /    \                                         /
     /    /       \                                       /
    /   /          \W5                                   /
   /  /             \                                   /
  / /                \                  __     h3      /
X2o----------------------------------  |   ------------o
           W6                        __|
</pre>


In [10]:
from math import *
import numpy as np

def nn1HiddenLayer3(X,W,showHidden=None,debug=False):
    z1 = X[0]*W[0]+X[1]*W[1];
    z2 = X[0]*W[2]+X[1]*W[3];
    z3 = X[0]*W[4]+X[1]*W[5];

    h1 = -1 if z1<0 else 1;
    h2 = -1 if z2<0 else 1;
    h3 = -1 if z3<0 else 1;
  
    z  = h1*W[6]+h2*W[7]+h3*W[8];
    return "-" if z<0 else "1";
    
W = [0,1,
     1,1,
     2,3,
     1,-2,1]

len(W)

9

In [11]:
scan2InputNN(nn1HiddenLayer3,W)

      -5  -4  -3  -2  -1  0  1  2  3  4  5  
	5  1  1  1  1  1  1  1  1  1  1  1  
	4  1  1  1  1  1  1  1  1  1  1  1  
	3  1  1  1  1  1  1  1  1  1  1  1  
	2  1  1  1  1  1  1  1  1  1  1  1  
	1  1  1  1  1  1  1  1  1  1  1  1  
	0  1  1  1  1  1  1  1  1  1  1  1  
	-1 1  1  1  1  1  1  -  -  -  -  -  
	-2 1  1  1  1  1  1  1  -  -  -  -  
	-3 1  1  1  1  1  1  1  1  -  -  -  
	-4 1  1  1  1  1  1  1  1  1  -  -  
	-5 1  1  1  1  1  1  1  1  1  1  -  
	

In [12]:
def nn1HiddenLayer3B(X,W,showHidden=None,debug=False):
    str=[False,False,False]
    h=0
    if showHidden : 
        h=showHidden-1
        assert(h>=0)
        str[h]=True 
        
    hl=[]
    hl.append( perceptron(X[0],X[1],W[0],W[1],str[0]) )
    hl.append( perceptron(X[0],X[1],W[2],W[3],str[1]) )
    hl.append( perceptron(X[0],X[1],W[4],W[5],str[2]) )
    
    if debug: print("h1: ",hl[0],"  h2: ",hl[1],"  h3: ",hl[2],"\nh: ",h,"  showHidden: ",showHidden)
        
    if showHidden: return hl[h]
        
    z = hl[0]*W[6]+hl[1]*W[7]
    
    if debug: print("S^{-1}(y): ",z+hl[2]*W[8],"\nW7 / W8 / W9 : ",W[6],W[7],W[8])
        
    return perceptron(z,hl[2],1,W[8],str=True)

W = [0,1,
     1,1,
     2,3,
     1,2,-3]

scan2InputNN(nn1HiddenLayer3B,W,xysup=21,showHidden=None)

      -10  -9  -8  -7  -6  -5  -4  -3  -2  -1  0  1  2  3  4  5  6  7  8  9  10  
	10  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
	9  -  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
	8  -  -  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
	7  -  -  -  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
	6  1  -  -  -  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
	5  1  1  1  -  -  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
	4  1  1  1  1  -  -  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
	3  1  1  1  1  1  1  -  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
	2  1  1  1  1  1  1  1  -  1  1  1  1  1  1  1  1  1  1  1  1  1  
	1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
	0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
	-1 1  1  1  1  1  1  1  1  1  1  1  1  -  -  -  -  -  -  -  -  -  
	-2 1  1  1  1  1  1  1  1  1  1  1  1  1  -  -  -  -  -  -  -  -  
	-3 1  1  1  1  1  1  1  1  1  1 

In [13]:
nn1HiddenLayer3B([3,2],W,showHidden=None,debug=True)

h1:  1   h2:  1   h3:  1 
h:  0   showHidden:  None
S^{-1}(y):  0 
W7 / W8 / W9 :  1 2 -3


'1'

In [14]:
perceptron(-5,0,0,1)

1

#### 3 Layers NN
<pre>
          W1                           __     h1
X1o---------------------------------  |   ------o
   \ \              /               __|          \
    \  \           /W2                            \w7
     \   \        /                                \ 
      \    \     /                                  \            __       J1
       \     \  /                                    /--------  |   ------o
        \      /\                                   /         __|          \
         \    /  \ W3                              /w8                      \w11
          \  /    \                               /                          \
            /      \                    __     h2/                            \        __            
           /\       /----------------  |   -----o  ----------/     /----------------  |   ------------o y
          /  \     /                 __|         \                             /    __|              
         /    \   /W4                             \w9                         /
        /      \ /                                 \                         /w12
       /      / \                                   \             __        /
      /     /    \                                   /---------  |   ------o
     /    /       \                                 /          __|         J2
    /   /          \W5                             /w10       
   /  /             \                             /      
  / /                \                  __     h3/      
X2o----------------------------------  |   -----o
           W6                        __|
</pre>

In [15]:
def nn2HiddenLayer32(X,W,showHidden=None,debug=False):
    assert(len(W)==12)
    str=[False,False,False,False,False]
    h=0
    if showHidden : 
        h=showHidden-1 # h1-3: 0-2 ; j1-2:3,4
        assert(h>=0)
        str[h]=True 
        
    hl=[]
    hl.append(nPerceptron(X=X,W=W[0:2],str=str[0]) )
    hl.append(nPerceptron(X=X,W=W[2:4],str=str[1]) )
    hl.append(nPerceptron(X=X,W=W[4:6],str=str[2]) )
    
    if showHidden and h<3: return hl[h]
    
    jl=[]
    jl.append(nPerceptron(X=[hl[0],hl[1]],W=W[6:8],str=str[3]) )
    jl.append(nPerceptron(X=[hl[1],hl[2]],W=W[8:10],str=str[4]) )

    if showHidden: return jl[h-3]  #only happens if not printing first hidden layer hl[h]

    if debug:
        for i in range(5):
            if i<3:
                v = hl[i]
                idx = i
                lbl='h'
            else:
                v = jl[i-3]
                idx = i-3
                lbl='J'
            print(lbl,"",idx,": ",v,"  ",end="",sep="")
        print("\nh: ",h,"  showHidden: ",showHidden,sep="")

    if showHidden: return hl[h] if h<3 else jl[h-3]
    
    return nPerceptron(X=[jl[0],jl[1]],W=W[10:12],str=True)


In [16]:
weights2Layers=[0,1,       #w1,w2
                1,1,       #w3,w4
                2,3,       #w5,w6
                -0.5,-.2,-.1,0.5, #w7,w8,w9,w10
                1,1]       #w11,12

#len([1,23,3])
#len(weights2Layers)
#nn2HiddenLayer32([3,3],weights2Layers)
scan2InputNN(nn2HiddenLayer32,weights=weights2Layers,showHidden=None,xysup=11)

      -5  -4  -3  -2  -1  0  1  2  3  4  5  
	5  1  1  1  1  1  1  1  1  1  1  1  
	4  1  1  1  1  1  1  1  1  1  1  1  
	3  -  1  1  1  1  1  1  1  1  1  1  
	2  -  -  1  1  1  1  1  1  1  1  1  
	1  -  -  -  -  1  1  1  1  1  1  1  
	0  -  -  -  -  -  1  1  1  1  1  1  
	-1 1  1  1  1  1  1  1  1  1  1  1  
	-2 1  1  1  1  1  1  1  1  1  1  1  
	-3 1  1  1  1  1  1  1  1  1  1  1  
	-4 1  1  1  1  1  1  1  1  1  1  1  
	-5 1  1  1  1  1  1  1  1  1  1  1  
	

## How?

In [17]:
scan2InputNN(nn2HiddenLayer32,weights=weights2Layers,showHidden=1,xysup=11)

Showing hidden neuron  1
      -5  -4  -3  -2  -1  0  1  2  3  4  5  
	5  1  1  1  1  1  1  1  1  1  1  1  
	4  1  1  1  1  1  1  1  1  1  1  1  
	3  1  1  1  1  1  1  1  1  1  1  1  
	2  1  1  1  1  1  1  1  1  1  1  1  
	1  1  1  1  1  1  1  1  1  1  1  1  
	0  1  1  1  1  1  1  1  1  1  1  1  
	-1 -  -  -  -  -  -  -  -  -  -  -  
	-2 -  -  -  -  -  -  -  -  -  -  -  
	-3 -  -  -  -  -  -  -  -  -  -  -  
	-4 -  -  -  -  -  -  -  -  -  -  -  
	-5 -  -  -  -  -  -  -  -  -  -  -  
	

In [18]:
scan2InputNN(nn2HiddenLayer32,weights=weights2Layers,showHidden=2,xysup=11)

Showing hidden neuron  2
      -5  -4  -3  -2  -1  0  1  2  3  4  5  
	5  1  1  1  1  1  1  1  1  1  1  1  
	4  -  1  1  1  1  1  1  1  1  1  1  
	3  -  -  1  1  1  1  1  1  1  1  1  
	2  -  -  -  1  1  1  1  1  1  1  1  
	1  -  -  -  -  1  1  1  1  1  1  1  
	0  -  -  -  -  -  1  1  1  1  1  1  
	-1 -  -  -  -  -  -  1  1  1  1  1  
	-2 -  -  -  -  -  -  -  1  1  1  1  
	-3 -  -  -  -  -  -  -  -  1  1  1  
	-4 -  -  -  -  -  -  -  -  -  1  1  
	-5 -  -  -  -  -  -  -  -  -  -  1  
	

In [19]:
scan2InputNN(nn2HiddenLayer32,weights=weights2Layers,showHidden=3,xysup=11)

Showing hidden neuron  3
      -5  -4  -3  -2  -1  0  1  2  3  4  5  
	5  1  1  1  1  1  1  1  1  1  1  1  
	4  1  1  1  1  1  1  1  1  1  1  1  
	3  -  1  1  1  1  1  1  1  1  1  1  
	2  -  -  1  1  1  1  1  1  1  1  1  
	1  -  -  -  -  1  1  1  1  1  1  1  
	0  -  -  -  -  -  1  1  1  1  1  1  
	-1 -  -  -  -  -  -  -  1  1  1  1  
	-2 -  -  -  -  -  -  -  -  1  1  1  
	-3 -  -  -  -  -  -  -  -  -  -  1  
	-4 -  -  -  -  -  -  -  -  -  -  -  
	-5 -  -  -  -  -  -  -  -  -  -  -  
	

In [20]:
scan2InputNN(nn2HiddenLayer32,weights=weights2Layers,showHidden=4,xysup=11)

Showing hidden neuron  4
      -5  -4  -3  -2  -1  0  1  2  3  4  5  
	5  -  -  -  -  -  -  -  -  -  -  -  
	4  -  -  -  -  -  -  -  -  -  -  -  
	3  -  -  -  -  -  -  -  -  -  -  -  
	2  -  -  -  -  -  -  -  -  -  -  -  
	1  -  -  -  -  -  -  -  -  -  -  -  
	0  -  -  -  -  -  -  -  -  -  -  -  
	-1 1  1  1  1  1  1  1  1  1  1  1  
	-2 1  1  1  1  1  1  1  1  1  1  1  
	-3 1  1  1  1  1  1  1  1  1  1  1  
	-4 1  1  1  1  1  1  1  1  1  1  1  
	-5 1  1  1  1  1  1  1  1  1  1  1  
	

In [21]:
scan2InputNN(nn2HiddenLayer32,weights=weights2Layers,showHidden=5,xysup=11)

Showing hidden neuron  5
      -5  -4  -3  -2  -1  0  1  2  3  4  5  
	5  1  1  1  1  1  1  1  1  1  1  1  
	4  1  1  1  1  1  1  1  1  1  1  1  
	3  -  1  1  1  1  1  1  1  1  1  1  
	2  -  -  1  1  1  1  1  1  1  1  1  
	1  -  -  -  -  1  1  1  1  1  1  1  
	0  -  -  -  -  -  1  1  1  1  1  1  
	-1 -  -  -  -  -  -  -  1  1  1  1  
	-2 -  -  -  -  -  -  -  -  1  1  1  
	-3 -  -  -  -  -  -  -  -  -  -  1  
	-4 -  -  -  -  -  -  -  -  -  -  -  
	-5 -  -  -  -  -  -  -  -  -  -  -  
	

## XOR
<pre>
          W1                    __     h1
X1o--------------------------  |   -----------o
   \                /        __|               \
    \              /W2                          \
     \            /                              \ 
      \          /                                \
       \        /                                  \
        \      /                                    \w5
         \    /                                      \
          \  /                                        \
            /                                          \          __            
           /\                                          --------  |  -------o y
          /  \                                         /       __|              
         /    \                                       /
        /      \                                     /
       /        \                                   /w6
      /          \                                 /
     /            \                               /
    /              \W3                           /
   /                \                           /
  /                  \          __     h2      /
X2o--------------------------  |   -----------o
           W4                __|



        X2 |                            h2  |
           |                                |
           1  0            ====>         ?  |  ?
           |                                |
   --------0--1----- X1             ---------------- h1
           |                                |
           |                             ?  |  ?
           |                                |
           |                                |
</pre>


In [22]:

def nonLinearHeaviside(x,str=None):
    if not str: return (-1 if x<0 else 1 )
    else: return ('-' if x<0 else '1')

def nonLinearRelu(x,str=None):
    r = max(0,x)
    if not str: return r
    else: return ''+r

def nn1HiddenLayer2(X,W,showHidden=None,debug=False):
    assert(len(W)==6)
    str=[False,False,False,False,False]
    h=0
    if showHidden : 
        h=(showHidden-1)%2 # h1-2
        assert(h>=0)
        str[h]=True 
        
    hl=[]
    hl.append(nPerceptron(X=X,W=W[0:2],str=str[0]) )
    hl.append(nPerceptron(X=X,W=W[2:4],str=str[1]) )

    if showHidden: return hl[h]
    
    return nPerceptron(X=[hl[0],hl[1]],W=W[4:6],str=True)

In [31]:
weights1Layer=[-1,1,
              1,-1,
              -1,-1
             ]
scan2InputNN(nn1HiddenLayer2,weights=weights1Layer,showHidden=None,xysup=3)

      -1  0  1  
	1  1  1  -  
	0  1  -  1  
	-1 -  1  1  
	

### Analysis

<pre>
XOR(o) = 1  |     No lime                                 No line 
XOR(x) = 0  o x   distinguish                        |    distinguish                  
            |     o from x         W               o |    o from x
      ______x_o____             ------->        _____x____
            |                                        | o          
            |                                        |             
            |                \                       | 
                              \
                               \  S ° W 
            |                   \                 \  |       Infinite lines
            | S                   \                o | x     distinguish o from x
            |                       \------>   _____\|_____  e.g., 2nd diagonal!
            v                                        |\
                                                     | o
                    No line                          |  \
            | ®     distinguish
       _____|_____  o from x
            |
            |
            
      Neither the linear transformation W nor the non-linear step function S alone allow us 
      to distinguish the two input cases of the XOR function. However, combining both yields
      a separable set.
</pre>

We can write the whole network as 

$y\,=\,S\left( \mathbf{\overrightarrow{w'}}\cdot \mathbf{\overrightarrow{h}} \right)\,=\,S\left((w_5,\,w_6)\cdot\begin{pmatrix}h_1\\h_2\end{pmatrix}\right)$  
$\mathbf{\overrightarrow{h}}\,=\,S\left(\mathbf{W}\,\mathbf{\overrightarrow{x}}\right)$

$S\big( (a,\,b)\big)\,\equiv\,\big((S(a),\,S(b))\big)$ and $S(a)\,=\,\left\{\begin{array} \,-1\;\mbox{if a<0}\\1\;\mbox{otherwise}\end{array}\right.$   
<pre>
            |             Properties of S:     \  |  / Leaves invariant both diagonals
           1|__ S(a)                            \ | /            |       
       _____|______                         _____\|/______     x | +  Maps origin and positive base vec -> (1,1)
          __|-1                                  /|\         __^_o_^__  
            |                                   / | \            |›x  Maps negative base vec onto 2nd diagonal  
            |                                  /  |  \           | 
            
</pre>
                                                 
with function $S$ being the (non-linear, Heaviside) step function. Also,

$\mathbf{W}\,\mathbf{\overrightarrow{x}}\,=\,\begin{bmatrix}-1 & 1 \\1 & -1\end{bmatrix}\,\begin{pmatrix}x_1\\x_2\end{pmatrix} $

and 

$\mathbf{\overrightarrow{x}}\,=\,\left\{\begin{array}\,(1,0)^t \\ (0,1)^t\end{array}\right. \quad\Rightarrow\quad W\mathbf{\overrightarrow{x}}\,=\,\left\{\begin{array}\,-(1,-1)^t\\-(-1,1)^t\end{array}\right.$

$\mathbf{\overrightarrow{x}}\,=\,\left\{\begin{array}\,(1,1)^t \\ (0,0)^t\end{array}\right. \quad\Rightarrow\quad W\mathbf{\overrightarrow{x}}\,=\,(0,0)$

Thus, the linear transformation $\mathbf{W}$ maps all four inputs **interspersed** along the second diagonal. On the other hand, if we'd feed the input directly to the non-linear step function $S$, all four input values would be mapped onto the single point $(1,1)$. Whence, in either case a projection onto any direction will not be able to untangle the two input cases of the XOR function.

However, it works if we combine both! Let's see how. First $\mathbf{W}$ maps the input cases onto the second diagonal and then the step function $S$ pulls the origin $(0,0$ off that diagonal and maps it onto point $(1,1)$. A straight line can then be used to distinguish them.


**Question: Which line could we then use to actually classify the two input cases of the XOR function?** Ans: There is of course an infinite number of lines that do the job. Among them we have, e.g., the 2nd diagonal $y=-x$, but also any othe parallel to that cutting through the first quadrant but leaving point $(1,1)$ still on its right side, i.e., $y=-x+b$ with $0\lt\,b\,\lt 2$.


<!--
#### Geometrical interpretation
The linear transformation given by $\mathbf{W}$ can be seen as $\mathbb{-1}\,+\,\mathbf{\sigma_x}$, where $\sigma_x\,=\,\begin{bmatrix}0&1\\1&0\end{bmatrix}\,=\,R_{-\pi/2}\cdot m_y$, i.e., a reflection over the y-axis $m_y$ (inverting the orientation of the x-axis) followed by a clock-wise rotation of $90\deg$, $R_{-\pi/2}$. Furthermore, the first term of $\mathbf{W}$, which corresponds to an inversion over the origin, can be constructed as a reflection over $y$-axis followed by another over the $x$-axis, i.e., $\mathbb{-1}\,=\,m_x\,m_y$. Whence, it is
$$\mathbf{W}\,=\,\left[m_x\,+\,R_{-\pi/2}\right]\,m_y$$

This leads to  the mapping $\overrightarrow{x}\,\to\,-\overrightarrow{x}\,+\,\overrightarrow{x'}$ where $\overrightarrow{x'}\,=\,(x_2,x_1)$. In terms of the reflections, $\overrightarrow{x}\,\to\,-\,(1\,-\,R_{-\pi/2}\,m_y)\,\overrightarrow{x}$

-->